### Combining NAAF-corrected *de novo* and PeaksDB  peptides for Trocas 7 incubation samples

The dataset:

    16 treatments: 4 stations, 2 timepoints (Time 0 an Time 24 hrs), 2 size fractions (GFF; GF75):
    
    Stations: 
    
        - Macapa South (MS) South stem, upriver
        - Macapa North (MN) North stem, upriver
        - Chaves (CV) South stem, downriver
        - Baylique (BY) North stem, downriver


    Proteomics samples from 2 trips to UWPR (July 2020 on the QE; April 2021 on the Fusion)
    There were at least triplicate samples for each treatment
    Many in April 2021 injected twice

Starting with:

    Peaks de novo results of PTM-optimized sequencing, NAAF corrected per sample
    PeaksDB de novo-assisted results from PTM-optimized database searches, NAAF corrected per sample
    
    Multiple samples per treatment

Goal:

    CSVs with combined de novo and PeaksDB for each sample, normalized to by NAAF
    
Using:

    - pandas
    - matplotlib
    - numpy

In [1]:
# LIBRARIES
#import pandas library for working with tabular data
import os
os.getcwd()
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from scipy.stats import kde
#import regular expresson (regex)
import re
#check pandas version
pd.__version__

'1.0.5'

In [2]:
cd /home/millieginty/Documents/git-repos/amazon/data/Trocas7-incubations/

/home/millieginty/Documents/git-repos/amazon/data/Trocas7-incubations


### 1. Macapa South, timepoint 0, size fraction 0.3-0.7 um (denoted as MS_T00_GF)
### T7 samples #s: 109, 110
### Exported NAAF and stripped peptides contained in the following directories:

    Trocas-incubations/processed/PeaksDB/110_MS_T00_GF_PDB
    Trocas-incubations/processed/PeaksDB/110_MS_T00_GF_DN

In [10]:
# for each of the 4 incubation samples:
# read in NAAF totals csvs made in PeaksDN (de novo), and PeaksDB (de novo assisted database search) notebooks
# bringing de novo peptides >50% ALC and PeaksDB peptides <1% FDR

peaks50_110a = pd.read_csv("processed/PeaksDN/110_MS_T00_GF_DN/110_MS_T00_GF_DN50_naaf_totals.csv")
peaks50_110b = pd.read_csv("processed/PeaksDN/110_MS_T00_GF_DN/110B_MS_T00_GF_DN50_naaf_totals.csv")
peaksdb_110a = pd.read_csv("processed/PeaksDB/110_MS_T00_GF_PDB/110_MS_T00_GF_PDB_naaf_totals.csv")
peaksdb_110b = pd.read_csv("processed/PeaksDB/110_MS_T00_GF_PDB/110B_MS_T00_GF_PDB_naaf_totals.csv")

frames = [peaks50_110a, peaks50_110b, peaksdb_110a, peaksdb_110b]
index = ['peaks50_110a', 'peaks50_110b', 'peaksdb_110a', 'peaksdb_110b']

# concatenate dataframes
tot_110 = pd.concat(frames, sort=False, ignore_index=True)

# there's a useless column in there
tot_110 = tot_110.loc[:, ~tot_110.columns.str.contains('^Unnamed')]

# reindex with peaks 
names = ['peaks50_110', 'peaks50_109', 'peaksdb_110', 'peaksdb_109']
tot_110.insert(loc=0, column='data source', value=names)
tot_110.set_index('data source')

# sum the AAs and PTMs

index = ['110']

data = {'NAAF': tot_110['NAAF'].sum(),
        'A-NAAF': tot_110['A'].sum(),
        'C-NAAF': tot_110['C'].sum(),
        'D-NAAF': tot_110['D'].sum(),
        'E-NAAF': tot_110['E'].sum(),
        'F-NAAF': tot_110['F'].sum(),
        'G-NAAF': tot_110['G'].sum(),
        'H-NAAF': tot_110['H'].sum(),
        'I-NAAF': tot_110['I'].sum(),
        'K-NAAF': tot_110['K'].sum(),
        'L-NAAF': tot_110['L'].sum(),
        'M-NAAF': tot_110['M'].sum(),
        'N-NAAF': tot_110['N'].sum(),
        'P-NAAF': tot_110['P'].sum(),
        'Q-NAAF': tot_110['Q'].sum(),
        'R-NAAF': tot_110['R'].sum(),
        'S-NAAF': tot_110['S'].sum(),
        'T-NAAF': tot_110['T'].sum(),
        'V-NAAF': tot_110['V'].sum(),
        'W-NAAF': tot_110['W'].sum(),
        'Y-NAAF': tot_110['Y'].sum(),
        'C-carb-NAAF': tot_110['c-carb'].sum(),
        'M-oxid-NAAF': tot_110['m-oxid'].sum(),
        'N-deam-NAAF': tot_110['n-deam'].sum(),
        'Q-deam-NAAF': tot_110['q-deam'].sum()
       }

sum_110 = pd.DataFrame(data, columns=['NAAF', 'A-NAAF', 'C-NAAF', 'D-NAAF', 'E-NAAF', 'F-NAAF', \
                                                   'G-NAAF', 'H-NAAF', 'I-NAAF','K-NAAF', 'L-NAAF', 'M-NAAF', \
                                                   'N-NAAF', 'P-NAAF', 'Q-NAAF', 'R-NAAF', 'S-NAAF', \
                                                   'T-NAAF', 'V-NAAF', 'W-NAAF', 'Y-NAAF', 'C-carb-NAAF', \
                                                   'M-oxid-NAAF', 'N-deam-NAAF', 'Q-deam-NAAF'], index=index)

# calculate the NAAF-corrected % modified C, M, N, Q
# also calculate the NAAF-corrected % of 'total modifiable AA space'
# this is a weighted average of modified residues wrt their relative abundance in the combined peptide set

sum_110['AA-NAAF'] = sum_110['A-NAAF'] + sum_110['C-NAAF'] + sum_110['D-NAAF'] + sum_110['E-NAAF'] + \
                     sum_110['F-NAAF'] + sum_110['G-NAAF'] + sum_110['H-NAAF'] + sum_110['I-NAAF'] + \
                     sum_110['K-NAAF'] + sum_110['L-NAAF'] + sum_110['M-NAAF'] + sum_110['N-NAAF'] + \
                     sum_110['P-NAAF'] + sum_110['Q-NAAF'] + sum_110['R-NAAF'] + sum_110['S-NAAF'] + \
                     sum_110['T-NAAF'] + sum_110['V-NAAF'] + sum_110['W-NAAF'] + sum_110['Y-NAAF'] 

sum_110['AA-modifiable-NAAF'] = sum_110['M-NAAF'] + sum_110['N-NAAF'] + sum_110['Q-NAAF']
                                

sum_110['Cys. w/ carb.'] = sum_110['C-carb-NAAF'] / sum_110['C-NAAF'] #1
sum_110['Met. w/ oxid.'] = sum_110['M-oxid-NAAF'] / sum_110['M-NAAF'] #2
sum_110['Asn. w/ deam.'] = sum_110['N-deam-NAAF'] / sum_110['N-NAAF'] #3
sum_110['Glu. w/ deam.'] = sum_110['Q-deam-NAAF'] / sum_110['Q-NAAF'] #3


sum_110['Overall modified'] = 0.001*((sum_110['Met. w/ oxid.']*sum_110['M-NAAF']) + \
                                     (sum_110['Asn. w/ deam.']*sum_110['N-NAAF']) + \
                                     (sum_110['Glu. w/ deam.']*sum_110['Q-NAAF']) / (sum_110['AA-modifiable-NAAF'])) 
                                                            

# write to a csv T7 combined data directory in /processed
sum_110.to_csv("processed/NAAF-sums/NAAF-sum-110.csv")

sum_110.head()

,NAAF,A-NAAF,C-NAAF,D-NAAF,E-NAAF,F-NAAF,G-NAAF,H-NAAF,I-NAAF,K-NAAF,...,M-oxid-NAAF,N-deam-NAAF,Q-deam-NAAF,AA-NAAF,AA-modifiable-NAAF,Cys. w/ carb.,Met. w/ oxid.,Asn. w/ deam.,Glu. w/ deam.,Overall modified
110,1041.599843,859.878964,196.519166,154.584513,138.88588,51.320592,198.524143,85.689246,39.364079,78.432194,...,93.393402,37.13559,2.978142,9909.078881,579.096024,1.0,0.686575,0.094981,0.057176,0.130534


### 2. Macapa South, timepoint 0, size fraction>0.7 um (denoted as MS_T00_GD)
### T7 samples #s: 209, 210
### Exported NAAF and stripped peptides contained in the following directories:

    Trocas-incubations/processed/PeaksDB/210_MS_T00_GD_PDB
    Trocas-incubations/processed/PeaksDB/210_MS_T00_GD_DN

In [11]:
# for each of the 4 incubation samples:
# read in NAAF totals csvs made in PeaksDN (de novo), and PeaksDB (de novo assisted database search) notebooks
# bringing de novo peptides >50% ALC and PeaksDB peptides <1% FDR

peaks50_210a = pd.read_csv("processed/PeaksDN/210_MS_T00_GD_DN/210_MS_T00_GD_DN50_naaf_totals.csv")
peaks50_210b = pd.read_csv("processed/PeaksDN/210_MS_T00_GD_DN/210B_MS_T00_GD_DN50_naaf_totals.csv")
peaksdb_210a = pd.read_csv("processed/PeaksDB/210_MS_T00_GD_PDB/210_MS_T00_GD_PDB_naaf_totals.csv")
peaksdb_210b = pd.read_csv("processed/PeaksDB/210_MS_T00_GD_PDB/210B_MS_T00_GD_PDB_naaf_totals.csv")

frames = [peaks50_210a, peaks50_210b, peaksdb_210a, peaksdb_210b]
index = ['peaks50_210a', 'peaks50_210b', 'peaksdb_210a', 'peaksdb_210b']

# concatenate dataframes
tot_210 = pd.concat(frames, sort=False, ignore_index=True)

# there's a useless column in there
tot_210 = tot_210.loc[:, ~tot_210.columns.str.contains('^Unnamed')]

# reindex with peaks 
names = ['peaks50_210', 'peaks50_209', 'peaksdb_210', 'peaksdb_209']
tot_210.insert(loc=0, column='data source', value=names)
tot_210.set_index('data source')

# sum the AAs and PTMs

index = ['210']

data = {'NAAF': tot_210['NAAF'].sum(),
        'A-NAAF': tot_210['A'].sum(),
        'C-NAAF': tot_210['C'].sum(),
        'D-NAAF': tot_210['D'].sum(),
        'E-NAAF': tot_210['E'].sum(),
        'F-NAAF': tot_210['F'].sum(),
        'G-NAAF': tot_210['G'].sum(),
        'H-NAAF': tot_210['H'].sum(),
        'I-NAAF': tot_210['I'].sum(),
        'K-NAAF': tot_210['K'].sum(),
        'L-NAAF': tot_210['L'].sum(),
        'M-NAAF': tot_210['M'].sum(),
        'N-NAAF': tot_210['N'].sum(),
        'P-NAAF': tot_210['P'].sum(),
        'Q-NAAF': tot_210['Q'].sum(),
        'R-NAAF': tot_210['R'].sum(),
        'S-NAAF': tot_210['S'].sum(),
        'T-NAAF': tot_210['T'].sum(),
        'V-NAAF': tot_210['V'].sum(),
        'W-NAAF': tot_210['W'].sum(),
        'Y-NAAF': tot_210['Y'].sum(),
        'C-carb-NAAF': tot_210['c-carb'].sum(),
        'M-oxid-NAAF': tot_210['m-oxid'].sum(),
        'N-deam-NAAF': tot_210['n-deam'].sum(),
        'Q-deam-NAAF': tot_210['q-deam'].sum()
       }

sum_210 = pd.DataFrame(data, columns=['NAAF', 'A-NAAF', 'C-NAAF', 'D-NAAF', 'E-NAAF', 'F-NAAF', \
                                                   'G-NAAF', 'H-NAAF', 'I-NAAF','K-NAAF', 'L-NAAF', 'M-NAAF', \
                                                   'N-NAAF', 'P-NAAF', 'Q-NAAF', 'R-NAAF', 'S-NAAF', \
                                                   'T-NAAF', 'V-NAAF', 'W-NAAF', 'Y-NAAF', 'C-carb-NAAF', \
                                                   'M-oxid-NAAF', 'N-deam-NAAF', 'Q-deam-NAAF'], index=index)

# calculate the NAAF-corrected % modified C, M, N, Q
# also calculate the NAAF-corrected % of 'total modifiable AA space'
# this is a weighted average of modified residues wrt their relative abundance in the combined peptide set

sum_210['AA-NAAF'] = sum_210['A-NAAF'] + sum_210['C-NAAF'] + sum_210['D-NAAF'] + sum_210['E-NAAF'] + \
                     sum_210['F-NAAF'] + sum_210['G-NAAF'] + sum_210['H-NAAF'] + sum_210['I-NAAF'] + \
                     sum_210['K-NAAF'] + sum_210['L-NAAF'] + sum_210['M-NAAF'] + sum_210['N-NAAF'] + \
                     sum_210['P-NAAF'] + sum_210['Q-NAAF'] + sum_210['R-NAAF'] + sum_210['S-NAAF'] + \
                     sum_210['T-NAAF'] + sum_210['V-NAAF'] + sum_210['W-NAAF'] + sum_210['Y-NAAF'] 

sum_210['AA-modifiable-NAAF'] = sum_210['M-NAAF'] + sum_210['N-NAAF'] + sum_210['Q-NAAF']
                                

sum_210['Cys. w/ carb.'] = sum_210['C-carb-NAAF'] / sum_210['C-NAAF'] #1
sum_210['Met. w/ oxid.'] = sum_210['M-oxid-NAAF'] / sum_210['M-NAAF'] #2
sum_210['Asn. w/ deam.'] = sum_210['N-deam-NAAF'] / sum_210['N-NAAF'] #3
sum_210['Glu. w/ deam.'] = sum_210['Q-deam-NAAF'] / sum_210['Q-NAAF'] #3


sum_210['Overall modified'] = 0.001*((sum_210['Met. w/ oxid.']*sum_210['M-NAAF']) + \
                                     (sum_210['Asn. w/ deam.']*sum_210['N-NAAF']) + \
                                     (sum_210['Glu. w/ deam.']*sum_210['Q-NAAF']) / (sum_210['AA-modifiable-NAAF'])) 
                                                            

# write to a csv T7 combined data directory in /processed
sum_210.to_csv("processed/NAAF-sums/NAAF-sum-210.csv")

sum_210.head()

,NAAF,A-NAAF,C-NAAF,D-NAAF,E-NAAF,F-NAAF,G-NAAF,H-NAAF,I-NAAF,K-NAAF,...,M-oxid-NAAF,N-deam-NAAF,Q-deam-NAAF,AA-NAAF,AA-modifiable-NAAF,Cys. w/ carb.,Met. w/ oxid.,Asn. w/ deam.,Glu. w/ deam.,Overall modified
210,1130.882117,891.34599,44.790374,70.798425,115.244996,42.215657,111.060918,71.096219,125.74765,83.907672,...,18.141832,45.934463,39.889628,9693.565499,333.73477,1.0,0.312601,0.230461,0.522221,0.064196


### 3. Macapa South, timepoint 24 hrs, size fraction 0.3-0.7 um (denoted as MS_T24_GF)
### T7 samples #s: 310A, 310B, 311 (310C)
### Exported NAAF and stripped peptides contained in the following directories:

    Trocas-incubations/processed/PeaksDB/310_MS_T24_GF_PDB
    Trocas-incubations/processed/PeaksDB/310_MS_T24_GF_DN

In [12]:
# for each of the 4 incubation samples:
# read in NAAF totals csvs made in PeaksDN (de novo), and PeaksDB (de novo assisted database search) notebooks
# bringing de novo peptides >50% ALC and PeaksDB peptides <1% FDR

peaks50_310a = pd.read_csv("processed/PeaksDN/310_MS_T24_GF_DN/310A_MS_T24_GF_DN50_naaf_totals.csv")
peaks50_310b = pd.read_csv("processed/PeaksDN/310_MS_T24_GF_DN/310B_MS_T24_GF_DN50_naaf_totals.csv")
peaks50_310c = pd.read_csv("processed/PeaksDN/310_MS_T24_GF_DN/310C_MS_T24_GF_DN50_naaf_totals.csv")
peaksdb_310a = pd.read_csv("processed/PeaksDB/310_MS_T24_GF_PDB/310A_MS_T24_GF_PDB_naaf_totals.csv")
peaksdb_310b = pd.read_csv("processed/PeaksDB/310_MS_T24_GF_PDB/310B_MS_T24_GF_PDB_naaf_totals.csv")
peaksdb_310c = pd.read_csv("processed/PeaksDB/310_MS_T24_GF_PDB/310C_MS_T24_GF_PDB_naaf_totals.csv")

frames = [peaks50_310a, peaks50_310b, peaks50_310c, peaksdb_310a, peaksdb_310b, peaksdb_310c]
index = ['peaks50_310a', 'peaks50_310b', 'peaks50_310c', 'peaksdb_310a', 'peaksdb_310b', 'peaksdb_310bc']

# concatenate dataframes
tot_310 = pd.concat(frames, sort=False, ignore_index=True)

# there's a useless column in there
tot_310 = tot_310.loc[:, ~tot_310.columns.str.contains('^Unnamed')]

# reindex with peaks 
names = ['peaks50_310a', 'peaks50_310b', 'peaks50_311', 'peaksdb_310a', 'peaksdb_310b', 'peaksdb_311']
tot_310.insert(loc=0, column='data source', value=names)
tot_310.set_index('data source')

# sum the AAs and PTMs

index = ['310']

data = {'NAAF': tot_310['NAAF'].sum(),
        'A-NAAF': tot_310['A'].sum(),
        'C-NAAF': tot_310['C'].sum(),
        'D-NAAF': tot_310['D'].sum(),
        'E-NAAF': tot_310['E'].sum(),
        'F-NAAF': tot_310['F'].sum(),
        'G-NAAF': tot_310['G'].sum(),
        'H-NAAF': tot_310['H'].sum(),
        'I-NAAF': tot_310['I'].sum(),
        'K-NAAF': tot_310['K'].sum(),
        'L-NAAF': tot_310['L'].sum(),
        'M-NAAF': tot_310['M'].sum(),
        'N-NAAF': tot_310['N'].sum(),
        'P-NAAF': tot_310['P'].sum(),
        'Q-NAAF': tot_310['Q'].sum(),
        'R-NAAF': tot_310['R'].sum(),
        'S-NAAF': tot_310['S'].sum(),
        'T-NAAF': tot_310['T'].sum(),
        'V-NAAF': tot_310['V'].sum(),
        'W-NAAF': tot_310['W'].sum(),
        'Y-NAAF': tot_310['Y'].sum(),
        'C-carb-NAAF': tot_310['c-carb'].sum(),
        'M-oxid-NAAF': tot_310['m-oxid'].sum(),
        'N-deam-NAAF': tot_310['n-deam'].sum(),
        'Q-deam-NAAF': tot_310['q-deam'].sum()
       }

sum_310 = pd.DataFrame(data, columns=['NAAF', 'A-NAAF', 'C-NAAF', 'D-NAAF', 'E-NAAF', 'F-NAAF', \
                                                   'G-NAAF', 'H-NAAF', 'I-NAAF','K-NAAF', 'L-NAAF', 'M-NAAF', \
                                                   'N-NAAF', 'P-NAAF', 'Q-NAAF', 'R-NAAF', 'S-NAAF', \
                                                   'T-NAAF', 'V-NAAF', 'W-NAAF', 'Y-NAAF', 'C-carb-NAAF', \
                                                   'M-oxid-NAAF', 'N-deam-NAAF', 'Q-deam-NAAF'], index=index)

# calculate the NAAF-corrected % modified C, M, N, Q
# also calculate the NAAF-corrected % of 'total modifiable AA space'
# this is a weighted average of modified residues wrt their relative abundance in the combined peptide set

sum_310['AA-NAAF'] = sum_310['A-NAAF'] + sum_310['C-NAAF'] + sum_310['D-NAAF'] + sum_310['E-NAAF'] + \
                     sum_310['F-NAAF'] + sum_310['G-NAAF'] + sum_310['H-NAAF'] + sum_310['I-NAAF'] + \
                     sum_310['K-NAAF'] + sum_310['L-NAAF'] + sum_310['M-NAAF'] + sum_310['N-NAAF'] + \
                     sum_310['P-NAAF'] + sum_310['Q-NAAF'] + sum_310['R-NAAF'] + sum_310['S-NAAF'] + \
                     sum_310['T-NAAF'] + sum_310['V-NAAF'] + sum_310['W-NAAF'] + sum_310['Y-NAAF'] 

sum_310['AA-modifiable-NAAF'] = sum_310['M-NAAF'] + sum_310['N-NAAF'] + sum_310['Q-NAAF']
                                

sum_310['Cys. w/ carb.'] = sum_310['C-carb-NAAF'] / sum_310['C-NAAF'] #1
sum_310['Met. w/ oxid.'] = sum_310['M-oxid-NAAF'] / sum_310['M-NAAF'] #2
sum_310['Asn. w/ deam.'] = sum_310['N-deam-NAAF'] / sum_310['N-NAAF'] #3
sum_310['Glu. w/ deam.'] = sum_310['Q-deam-NAAF'] / sum_310['Q-NAAF'] #3


sum_310['Overall modified'] = 0.001*((sum_310['Met. w/ oxid.']*sum_310['M-NAAF']) + \
                                     (sum_310['Asn. w/ deam.']*sum_310['N-NAAF']) + \
                                     (sum_310['Glu. w/ deam.']*sum_310['Q-NAAF']) / (sum_310['AA-modifiable-NAAF'])) 
                                                            

# write to a csv T7 combined data directory in /processed
sum_310.to_csv("processed/NAAF-sums/NAAF-sum-310.csv")

sum_310.head()

,NAAF,A-NAAF,C-NAAF,D-NAAF,E-NAAF,F-NAAF,G-NAAF,H-NAAF,I-NAAF,K-NAAF,...,M-oxid-NAAF,N-deam-NAAF,Q-deam-NAAF,AA-NAAF,AA-modifiable-NAAF,Cys. w/ carb.,Met. w/ oxid.,Asn. w/ deam.,Glu. w/ deam.,Overall modified
310,876.02962,633.973435,26.280746,9.994564,33.399379,77.576323,98.419832,138.118882,21.575185,106.613832,...,9.714406,1.641078,0.405193,7564.436075,228.927931,1.0,0.148148,0.010817,0.034806,0.011357


### 4. Macapa South, timepoint 24 hrs, size fraction >0.7 um (denoted as MS_T24_GD)
### T7 samples #s: 410A, 410B
### Exported NAAF and stripped peptides contained in the following directories:

    Trocas-incubations/processed/PeaksDB/410_MS_T24_GD_PDB
    Trocas-incubations/processed/PeaksDB/410_MS_T24_GD_DN

In [13]:
# for each of the 4 incubation samples:
# read in NAAF totals csvs made in PeaksDN (de novo), and PeaksDB (de novo assisted database search) notebooks
# bringing de novo peptides >50% ALC and PeaksDB peptides <1% FDR

peaks50_410a = pd.read_csv("processed/PeaksDN/410_MS_T24_GD_DN/410A_MS_T24_GD_DN50_naaf_totals.csv")
peaks50_410b = pd.read_csv("processed/PeaksDN/410_MS_T24_GD_DN/410B_MS_T24_GD_DN50_naaf_totals.csv")
peaksdb_410a = pd.read_csv("processed/PeaksDB/410_MS_T24_GD_PDB/410A_MS_T24_GD_PDB_naaf_totals.csv")
peaksdb_410b = pd.read_csv("processed/PeaksDB/410_MS_T24_GD_PDB/410B_MS_T24_GD_PDB_naaf_totals.csv")


frames = [peaks50_410a, peaks50_410b, peaksdb_410a, peaksdb_410b, ]
index = ['peaks50_410a', 'peaks50_410b', 'peaksdb_410a', 'peaksdb_410b']

# concatenate dataframes
tot_410 = pd.concat(frames, sort=False, ignore_index=True)

# there's a useless column in there
tot_410 = tot_410.loc[:, ~tot_410.columns.str.contains('^Unnamed')]

# reindex with peaks 
names = ['peaks50_410a', 'peaks50_410b', 'peaksdb_410a', 'peaksdb_410b']
tot_410.insert(loc=0, column='data source', value=names)
tot_410.set_index('data source')

# sum the AAs and PTMs

index = ['410']

data = {'NAAF': tot_410['NAAF'].sum(),
        'A-NAAF': tot_410['A'].sum(),
        'C-NAAF': tot_410['C'].sum(),
        'D-NAAF': tot_410['D'].sum(),
        'E-NAAF': tot_410['E'].sum(),
        'F-NAAF': tot_410['F'].sum(),
        'G-NAAF': tot_410['G'].sum(),
        'H-NAAF': tot_410['H'].sum(),
        'I-NAAF': tot_410['I'].sum(),
        'K-NAAF': tot_410['K'].sum(),
        'L-NAAF': tot_410['L'].sum(),
        'M-NAAF': tot_410['M'].sum(),
        'N-NAAF': tot_410['N'].sum(),
        'P-NAAF': tot_410['P'].sum(),
        'Q-NAAF': tot_410['Q'].sum(),
        'R-NAAF': tot_410['R'].sum(),
        'S-NAAF': tot_410['S'].sum(),
        'T-NAAF': tot_410['T'].sum(),
        'V-NAAF': tot_410['V'].sum(),
        'W-NAAF': tot_410['W'].sum(),
        'Y-NAAF': tot_410['Y'].sum(),
        'C-carb-NAAF': tot_410['c-carb'].sum(),
        'M-oxid-NAAF': tot_410['m-oxid'].sum(),
        'N-deam-NAAF': tot_410['n-deam'].sum(),
        'Q-deam-NAAF': tot_410['q-deam'].sum()
       }

sum_410 = pd.DataFrame(data, columns=['NAAF', 'A-NAAF', 'C-NAAF', 'D-NAAF', 'E-NAAF', 'F-NAAF', \
                                                   'G-NAAF', 'H-NAAF', 'I-NAAF','K-NAAF', 'L-NAAF', 'M-NAAF', \
                                                   'N-NAAF', 'P-NAAF', 'Q-NAAF', 'R-NAAF', 'S-NAAF', \
                                                   'T-NAAF', 'V-NAAF', 'W-NAAF', 'Y-NAAF', 'C-carb-NAAF', \
                                                   'M-oxid-NAAF', 'N-deam-NAAF', 'Q-deam-NAAF'], index=index)

# calculate the NAAF-corrected % modified C, M, N, Q
# also calculate the NAAF-corrected % of 'total modifiable AA space'
# this is a weighted average of modified residues wrt their relative abundance in the combined peptide set

sum_410['AA-NAAF'] = sum_410['A-NAAF'] + sum_410['C-NAAF'] + sum_410['D-NAAF'] + sum_410['E-NAAF'] + \
                     sum_410['F-NAAF'] + sum_410['G-NAAF'] + sum_410['H-NAAF'] + sum_410['I-NAAF'] + \
                     sum_410['K-NAAF'] + sum_410['L-NAAF'] + sum_410['M-NAAF'] + sum_410['N-NAAF'] + \
                     sum_410['P-NAAF'] + sum_410['Q-NAAF'] + sum_410['R-NAAF'] + sum_410['S-NAAF'] + \
                     sum_410['T-NAAF'] + sum_410['V-NAAF'] + sum_410['W-NAAF'] + sum_410['Y-NAAF'] 

sum_410['AA-modifiable-NAAF'] = sum_410['M-NAAF'] + sum_410['N-NAAF'] + sum_410['Q-NAAF']
                                

sum_410['Cys. w/ carb.'] = sum_410['C-carb-NAAF'] / sum_410['C-NAAF'] #1
sum_410['Met. w/ oxid.'] = sum_410['M-oxid-NAAF'] / sum_410['M-NAAF'] #2
sum_410['Asn. w/ deam.'] = sum_410['N-deam-NAAF'] / sum_410['N-NAAF'] #3
sum_410['Glu. w/ deam.'] = sum_410['Q-deam-NAAF'] / sum_410['Q-NAAF'] #3


sum_410['Overall modified'] = 0.001*((sum_410['Met. w/ oxid.']*sum_410['M-NAAF']) + \
                                     (sum_410['Asn. w/ deam.']*sum_410['N-NAAF']) + \
                                     (sum_410['Glu. w/ deam.']*sum_410['Q-NAAF']) / (sum_410['AA-modifiable-NAAF'])) 
                                                            

# write to a csv T7 combined data directory in /processed
sum_410.to_csv("processed/NAAF-sums/NAAF-sum-410.csv")

sum_410.head()

,NAAF,A-NAAF,C-NAAF,D-NAAF,E-NAAF,F-NAAF,G-NAAF,H-NAAF,I-NAAF,K-NAAF,...,M-oxid-NAAF,N-deam-NAAF,Q-deam-NAAF,AA-NAAF,AA-modifiable-NAAF,Cys. w/ carb.,Met. w/ oxid.,Asn. w/ deam.,Glu. w/ deam.,Overall modified
410,940.266393,719.044837,25.895355,28.780109,25.126785,30.970944,54.054334,47.438131,49.197527,37.744157,...,9.591568,6.019588,2.13739,7975.526648,239.479891,1.0,0.256142,0.031715,0.174783,0.01562
